#### ASTR288P Assignment 4 SOLUTIONS
Assigned: 2017/10/6 4PM

Due:      2017/10/20 2PM

## Coding Questions

### 1- Statistics! (20 points)

### a) Shot Noise (10 points)

We discussed briefly in class Gaussian and Poisson distributions. Shot noise (see [https://en.wikipedia.org/wiki/Shot_noise](https://en.wikipedia.org/wiki/Shot_noise)) is an example of random noise that is governed by a Poisson process. 

$$ P (N = k) = e^{-\lambda \Delta t} \frac{(\lambda \Delta t)^k}{k!} $$

The function defined below returns the probability $P$ of receiveing $k$ noise photons given a background photon rate of $\lambda$ photons/second and a time period $t$. 

The 'background' flux of the night sky can be modeled as Poissonian noise; under dark skies, the night sky background has a flux of roughly 
$$F_0 = 3\times10^{12}~\mathrm{photons/ s/ m^2/ sr}$$
in the regime relevant to ground-based gamma-ray telescopes. These telescopes look at very bright but very quick (10s of nanoseconds) long flashes of light. In order to 'trigger' on and save these flashes, these telescopes require a certain numbr of photons (the *trigger threshold*) to arrive within some period of time known as the *trigger window*. 

Assuming a collection area of $100~\mathrm{m}^2$, a field of view of $\sim 2\times10^{-6}~\mathrm{sr}$, and assuming that the telescope sensor detects only 30% of all photons, answer the following questions:

1. If the telescope trigger requires 5 photons to be seen in a 10ns window, what is the probability that the trigger will be random noise? 
1. Plot the probability as a function of number of photons ($k$) for three different night sky background values: $F_0$, $5F_0$, and $10F_0$ (corresponding to roughly new moon, and different amounts of moonlight). Plot your results from $k=0$ to $k=30$ photons.
1. Demonstrate that as you narrow the trigger window (i.e. require a smaller $\Delta t$ for arriving photons) the probability of getting a given number of photons decreases by plotting the Poisson probability as a function of trigger window. Do this for $k=3,5,10$. **Hint** This can be made simple by choosing a fixed trigger threshold and changing the trigger window $\Delta t$. 


**Make sure your plots have legends and plot axes so I know how to interpret them!**

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import factorial

def poisson(l,t,k):
    '''
    parameters: 
        l : flux
        t : time
        k : number of photons
    '''
    return np.exp(-l*t) * ((l*t)**k)/(factorial(k))

#this is the number of photons (our x-axis)
k = np.array(np.arange(0,30))
plt.plot(k, poisson(10*180e6,10e-9,k))
plt.plot(k, poisson(5*180e6,10e-9,k))
plt.plot(k, poisson(1*180e6,10e-9,k))
plt.show()

f = np.linspace(0,1,50)
plt.plot(f, poisson(5*180e6,f*10e-9,3))
plt.plot(f, poisson(5*180e6,f*10e-9,5))
plt.plot(f, poisson(5*180e6,f*10e-9,10))
plt.show()



##Place your code in this cell. Add more cells if you require them.


In [ ]:
#1
flux = 3e12 * 2e-6 * 100 * 0.3 #ph/m^2/sr/s * sr * m^2 
delta_t = 10e-9 #s
n_phot = 5
probability = poisson(flux, delta_t, 5)
print("The possibility of counting {0:d} photons in a {1:g} ns time window given a flux of {2:5.3g} photons/s is {3:.3f}".format(n_phot , delta_t * 1.e9, flux, probability))

In [ ]:
#2 
factors = [1., 5., 10.]

k = np.array(np.arange(0,30))
plt.figure(figsize=[8,6])
for factor in factors:
    plt.plot(k, poisson(factor * flux, delta_t, k), label="{0:g} $F_0$".format(factor))

plt.legend()
plt.ylabel("probability")
plt.xlabel("number of photons")
plt.show()

In [ ]:
#3
trigger_thresholds = [3, 5, 10]

trigger_window = np.linspace(10.e-9, 100.e-9, 50)

plt.figure(figsize=[8, 6])
for n_phot in trigger_thresholds:
    plt.plot(trigger_window*1.e9, poisson(flux, trigger_window, n_phot), label="k={0:d}".format(n_phot))
    
plt.xlabel("trigger window (ns)")
plt.ylabel("probability")
plt.legend()
plt.show()

#### b) Gaussian vs Poissonian (10 points)

Look up the definition of a Poisson and Gaussian distribution. Demonstrate that for large mean values, a Poisson distribution and Gaussian distribution look very similar. You will need to know that the standard deviation of a Poisson distribution is the square root of the mean in order to correctly define the Gaussian:
$$ \sigma = \sqrt{\mu} $$

Please demonstrate this in two ways: 
1. Use numpy.random like we used in class to generate random numbers to do so, and plot a histogram. You will need to draw a sufficiently large number of random numbers to not be stifled by statistics! Use 100 bins in your histogram.
1. Write a function that gives you a Gaussian distribution and then plot it alongside a Poisson distribution defined earlier.

In [ ]:
##Place your code in this cell. Add more cells if you require them.
#1. 
mean = 1000.
bins = np.linspace(750, 1250, 100)
poisson_vals = np.random.poisson(mean, size=25000)
plt.hist(poisson_vals, bins=bins, alpha=0.5, label="Poisson")
gaussian_vals = np.random.normal(mean, np.sqrt(mean), size=25000)
plt.hist(gaussian_vals, bins=bins, alpha=0.9, label="Gaussian",histtype='step')
plt.legend()
plt.xlabel("random number")
plt.show()

#2. 

def gaussian(x, *params):
    mu, sigma = params
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))

gaussian_mean = 12.*flux * delta_t
x = np.linspace(1,51,1000)
poisson_y = poisson(12.*flux, delta_t, x)
gaussian_y = gaussian(x, *[gaussian_mean, np.sqrt(gaussian_mean)])
gaussian_y = gaussian_y * poisson_y.max() #normalize the Gaussian and Poisson heights
plt.plot(x, poisson_y, label="Poission")
plt.plot(x, gaussian_y, label="Gaussian")
plt.legend()
plt.xlabel("measured value")
plt.show()

## 2- Slice and Dice (10 marks)

In this question we will practice slicing numpy arrays.
With the exception of creating and reshaping your initial arrays, you should not have to do any other assignments to complete the question, only slicing.

In [ ]:
import numpy as np
a = np.linspace(-12,12, 16)
print(a)
#A)
#Slice only the odd-indexed values in the array and print them to screen.
#For example, if our array is [2, 12, 20, -99, -57], then all odd-indexed values are 12 and -99.
### PLACE YOUR CODE BELOW THIS COMMENT
print("A:")
print(a[1::2])
print("")
a = np.linspace(-12,12, 21)
#B)
#Next, provide every 5th value starting with the third value in the array.
# HINT: Remember zero-indexing vs one-indexing!
#For example, if our array is 
#a = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
# the result is [2, 7, 12, 17, 22, 27]
### PLACE YOUR CODE BELOW THIS COMMENT
print("")
print("")
print("a = ", a)
print("B:")
print(a[2::5])
print("")

#C)
a = np.linspace(-12,12, 21)
#Slice from the 5th element to the 3rd-last element.
### PLACE YOUR CODE BELOW THIS COMMENT
print("C:")
print(a[4:-3])
print("")



#D)
print("D")
a = np.array(range(16))
a = a.reshape((4,4))
print(a)
#Slice out the first row and first column of the array so that ythe resulting 'view' of the array is
#[[5 6 7]
# [9 10 11]
# [13, 14 15]]
### PLACE YOUR CODE BELOW THIS COMMENT
print(a[1::, 1::])
print("")

#Provide code so that the resulting 'view' of the array is only the four 'middle' elements in the square. 
#E)
#[[5 6]
#[9 10]]
### PLACE YOUR CODE BELOW THIS COMMENT
print("E")
#Provide code that produces a 3x4x4 matrix (x,y,z coordinates) using the array(), range() and reshape() commands.
### PLACE YOUR CODE BELOW THIS COMMENT
a = np.array(range(3*4*4)).reshape(3,4,4)
print("a=", a)
#Now, produce a 3x3x2 view of the above matrix where 
#you omit the first row and the first and last column, but keep each 'layer' of the array.
print(a[:, 1:, 1:-1])